# Modifying the Bestiary

This is a good example of the code that you need to quickly fetch and modify data in S3. The cost of this is relatively low and S3 can `get` and `post` at internet speeds so it is very suitable for most applications. 

In [40]:
import pandas as pd
import numpy as np
import yaml
from boto.s3.connection import S3Connection

update_bestiary = False

### Load the values from S3 and transform to Pandas
This notebook is for bulk editing of the bestiary. Changing features of monsters changes how they are rendered and how they behave.  Once done I can upload the json back to S3 where it can be loaded into the live game. 

In [41]:
myKeys = yaml.load(open('/home/billmanh/Downloads/Key_file', 'r'))
AWSSecretKey=myKeys['AWSSecretKey']
AWSAccessKeyId=myKeys['AWSAccessKeyId']
conn = S3Connection(AWSAccessKeyId, AWSSecretKey)
mybucket = conn.get_bucket('flaskgame')

k = mybucket.get_key('terrain/beastiary')
y = yaml.load(k.get_contents_as_string())

These two steps are separated so that you can revert back to the origional dataset without having to re-fetch from S3. 

In [42]:
bdf = pd.DataFrame(y).T
bdf.head(10)

,attackType,color,damage,group max,group min,health,healthMaxVariance,move,move type,name,perception,render type,size
goblin,swung a rusty sword,#3E8E3E,4,10,5,3,2,10,swarm,Goblin,50,pack,5
raider,swung a daggar,#4a3b29,6,3,2,15,5,6,swarm,Raider,600,pack,10
scorpion,struck with it's stinger,#716D6F,12,2,1,8,7,3,swarm,Giant Scorpion,40,scatter,8
tiger,swung it's might claws,#e08d3c,12,1,1,8,4,20,swarm,Tiger,800,alone,6
yeti,swung with it's fists,#a0d6b4,8,1,1,15,3,5,swarm,Yeti,200,alone,15


## This section would be used to make the actual changes

I can also add default values to all monsters quickly, however the business logic should be set so that it never crashes due to the lack of an attribute.

This cell is the only one that I change. I just make all of the edits using simple pandas indexing commands unil I have what I want and then I upload the new values in the next step. 

In [43]:
#bdf['perception'] = bdf['perception']*4
bdf.loc['goblin']['perception'] = 50
bdf.loc['tiger']['move'] = 20
bdf

,attackType,color,damage,group max,group min,health,healthMaxVariance,move,move type,name,perception,render type,size
goblin,swung a rusty sword,#3E8E3E,4,10,5,3,2,10,swarm,Goblin,50,pack,5
raider,swung a daggar,#4a3b29,6,3,2,15,5,6,swarm,Raider,600,pack,10
scorpion,struck with it's stinger,#716D6F,12,2,1,8,7,3,swarm,Giant Scorpion,40,scatter,8
tiger,swung it's might claws,#e08d3c,12,1,1,8,4,20,swarm,Tiger,800,alone,6
yeti,swung with it's fists,#a0d6b4,8,1,1,15,3,5,swarm,Yeti,200,alone,15


## Uploading the Bestiary:

This is inside of an if statement so that you can set the default value above to `False` and avoid accidentally updating the database. 

In [36]:
if update_bestiary:
    k.set_contents_from_string(str(bdf.T.to_dict()))

The Bestiary was updated
